In [4]:
import os.path
from os import path
import pandas as pd


# 캐글 ↓제출경로
data_path = '/kaggle/input/bike-sharing-demand/'
if not path.exists(data_path):
    data_path = '../input/'
print(f'input directory path : {data_path}')

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

print(f'train shape : {train.shape}')
print(f'test_shape : {test.shape}')

# 날씨 weather ==> 4 인것 제거,
train = train[train['weather'] != 4]

# train 과 test 데이터 합치기 (index 역시 합쳐야 한다.)

all_data = pd.concat([train, test], ignore_index=True)
all_data

from datetime import datetime

# 날짜 피처 생성
all_data['date'] = all_data['datetime'].apply(lambda x : x.split()[0])

# 연도 피처
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])

# 월피처
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])

# 시 피처 생성
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# 요일 피처 생성
all_data['weekday'] = all_data['date'].apply(lambda dateString : datetime.strptime(dateString, "%Y-%m-%d").weekday())

# 훈련데이터 매달 1일~19일, 테스트 데이터 매달 20일~ 이므로 day 사용필요 x 
# minute, second 역시 사용필요 x (상식적)
all_data.head()


# all_data['datetime'] = pd.to_datetime(all_data['datetime'])
# all_data['year'] = all_data['datetime'].dt.year , dt.month, dt.hour, dt.weekday 로 변경가능


# 필요 없는 피처 제거

# 1. casual 과 registered 피처는 테스트 데이터 에 없는 피처이므로 제거
# 2. datetime 피처는  date(year,month,day) 피처가 가지고 있기에 필요 x
# 3. season 피처가 month 대분류이기에 month 제거
# 4. windspeed 상관관계 약하기에 제거 한다 

drop_features = ['casual', 'registered', 'datetime', 'date', 'month', 'windspeed']
all_data = all_data.drop(drop_features, axis=1)

all_data

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

# 타깃값 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

y = train['count'] # 타깃값 

X_train.head()

# 테스트 데이터 왜있는지....
# 평가 지표 계싼 함수 작성

import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    # 지수변환
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
        
    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))
    
    # RMSLE 계산
    output = np.sqrt(np.mean((log_true -log_pred)**2))
    return output

# sklearn.metrics.mean_squared_log_error 이용시 대체가능

input directory path : ../input/
train shape : (10886, 12)
test_shape : (6493, 9)


In [5]:
import numpy as np

# Generate some sample data
np_x_train = X_train.to_numpy().astype(float)

log_y = np.log(y)
np_log_y = log_y.to_numpy().reshape(len(log_y),1)
insert_column = np.ones_like(np_log_y)
np_x_train = np.c_[np_x_train, insert_column]

alpha = 0.1
max_iter = 3000

# Define the learning rate for gradient descent
eta = 0.001

# Initialize the weights with random values
weights = np.zeros((1,11))

# Define the cost function for Ridge regression
def ridge_cost(X, y, weights, alpha):
    # Calculate the mean squared error
    mse = np.mean((y - X.dot(weights)) ** 2)
    # Add the L2 regularization term
    l2_reg = alpha * np.sum(weights[:-1] ** 2)
    # Return the total cost
    return mse + l2_reg

# Define the gradient function for Ridge regression
def ridge_gradient(X, y, weights, alpha):
    # Calculate the residual
    residual = X.dot(weights) - y
    # Calculate the gradient for the weights
    grad = 2 * X.T.dot(residual)
    # Add the L2 regularization term
    grad[:-1] += 2 * alpha * weights[:-1]
    # Return the gradient
    return grad

# Perform gradient descent to optimize the weights
for i in range(max_iter):
    # Calculate the current cost
    cost = ridge_cost(np_x_train, np_log_y, weights, alpha)
    # Calculate the current gradient
    grad = ridge_gradient(np_x_train, np_log_y, weights, alpha)
    # Update the weights
    weights -= eta * grad
    # Print the current cost every 100 iterations
    if i % 100 == 0:
        print(f'Iteration {i}: cost={cost:.4f}')
        
# Make predictions on new data
X_new = np.random.rand(10, 5)
X_new = np.concatenate([X_new, np.ones((10, 1))], axis=1)
y_pred = X_new.dot(weights[:-1]) + weights[-1]

ValueError: shapes (10885,11) and (6,) not aligned: 11 (dim 1) != 6 (dim 0)